In [1]:
import pandas as pd

In [2]:
#Loading data from the Github repository to colab notebook
filename = 'https://raw.githubusercontent.com/PacktWorkshops/The-Data-Science-Workshop/master/Chapter15/Dataset/crx.data'



In [3]:
# Loading the data using pandas

credData = pd.read_csv(filename,sep=",",header = None,na_values = "?")
credData.head()

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15
0,b,30.83,0.000,u,g,w,v,1.25,t,t,1,f,g,202.0,0,+
1,a,58.67,4.460,u,g,q,h,3.04,t,t,6,f,g,43.0,560,+
2,a,24.50,0.500,u,g,q,h,1.50,t,f,0,f,g,280.0,824,+
3,b,27.83,1.540,u,g,w,v,3.75,t,t,5,t,g,100.0,3,+
4,b,20.17,5.625,u,g,w,v,1.71,t,f,0,f,s,120.0,0,+


In [4]:
# Changing the Classes to 1 & 0
credData.loc[credData[15] == '+' , 15] = 1
credData.loc[credData[15] == '-' , 15] = 0
credData.head()

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15
0,b,30.83,0.000,u,g,w,v,1.25,t,t,1,f,g,202.0,0,1
1,a,58.67,4.460,u,g,q,h,3.04,t,t,6,f,g,43.0,560,1
2,a,24.50,0.500,u,g,q,h,1.50,t,f,0,f,g,280.0,824,1
3,b,27.83,1.540,u,g,w,v,3.75,t,t,5,t,g,100.0,3,1
4,b,20.17,5.625,u,g,w,v,1.71,t,f,0,f,s,120.0,0,1


In [5]:
# Dropping all the rows with na values
newcred = credData.dropna(axis = 0)
newcred.shape

(653, 16)

In [6]:
# Seperating the categorical variables to make dummy variables

credCat = pd.get_dummies(newcred[[0,3,4,5,6,8,9,11,12]])


In [7]:
# Seperating the numerical variables

credNum = newcred[[1,2,7,10,13,14]]


In [8]:
# Making the X variable which is a concatenation of categorical and numerical data

X = pd.concat([credCat,credNum],axis = 1)
print(X.shape)

# Seperating the label as y variable
y = pd.Series(newcred[15], dtype="int")
print(y.shape)

(653, 46)
(653,)


In [9]:
# Normalising the data sets
# Import library function
from sklearn import preprocessing
# Creating the scaling function
minmaxScaler = preprocessing.MinMaxScaler()
# Transforming with the scaler function
X_tran = pd.DataFrame(minmaxScaler.fit_transform(X))
# Printing the output
X_tran.head()

,0,1,2,3,4,5,6,7,8,9,...,36,37,38,39,40,41,42,43,44,45
0,0.0,1.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,...,0.0,1.0,0.0,0.0,0.271111,0.000000,0.043860,0.014925,0.1010,0.00000
1,1.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,...,0.0,1.0,0.0,0.0,0.713016,0.159286,0.106667,0.089552,0.0215,0.00560
2,1.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,...,0.0,1.0,0.0,0.0,0.170635,0.017857,0.052632,0.000000,0.1400,0.00824
3,0.0,1.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,...,1.0,1.0,0.0,0.0,0.223492,0.055000,0.131579,0.074627,0.0500,0.00003
4,0.0,1.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,1.0,0.101905,0.200893,0.060000,0.000000,0.0600,0.00000


In [10]:
# Splitting the data set to train and test sets
from sklearn.model_selection import train_test_split

# Splitting the data into train and test sets
X_train, X_test, y_train, y_test = train_test_split(X_tran, y, test_size=0.3, random_state=123)



**MaxVoting**

In [11]:
# Defining the Voting classifier and three individual learners
from sklearn.ensemble import VotingClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier

# Defining the models
model1 = LogisticRegression(random_state=123)
model2 = KNeighborsClassifier(n_neighbors=5)
model3 = RandomForestClassifier(n_estimators=500)

In [12]:
# Defining the ensemble model using VotingClassifier
model = VotingClassifier(estimators=[('lr', model1),('knn', model2),('rf',model3)], voting='hard')

In [13]:
# Fitting the model on the training set
model.fit(X_train,y_train)

VotingClassifier(estimators=[('lr',
                              LogisticRegression(C=1.0, class_weight=None,
                                                 dual=False, fit_intercept=True,
                                                 intercept_scaling=1,
                                                 l1_ratio=None, max_iter=100,
                                                 multi_class='auto',
                                                 n_jobs=None, penalty='l2',
                                                 random_state=123,
                                                 solver='lbfgs', tol=0.0001,
                                                 verbose=0, warm_start=False)),
                             ('knn',
                              KNeighborsClassifier(algorithm='auto',
                                                   leaf_size=30,
                                                   metric='minkowski',
                                                 

In [14]:
# Predicting accuracy on the test set using .score() function
model.score(X_test,y_test)

0.9081632653061225

In [15]:
# Generating the predictions on the test set
preds = model.predict(X_test)

In [16]:
# Printing the confusion matrix
from sklearn.metrics import confusion_matrix
# Confusion matrix for the test set
print(confusion_matrix(y_test, preds))

[[96 11]
 [ 7 82]]


In [17]:
# Printing the classification report
from sklearn.metrics import classification_report
print(classification_report(y_test, preds))

              precision    recall  f1-score   support

           0       0.93      0.90      0.91       107
           1       0.88      0.92      0.90        89

    accuracy                           0.91       196
   macro avg       0.91      0.91      0.91       196
weighted avg       0.91      0.91      0.91       196

